In [1]:
pip install openturns
pip install otrobopt

     |████████████████████████████████| 48.0 MB 39 kB/s 


In [4]:
import otrobopt
import openturns as ot
from RBDO_class import *
import time
ZDist = ot.Normal(0,1)

# Comparison of RIA/SORA/PMA with ot-robopt

This notebook compares ot-robopt with the RBDO algorithms on a toy case


The RBDO problem to solve is the following


\begin{eqnarray}
\min_{d_1,d_2} & & (d_1-0.25)^2+d_2*d_1 \\
\text{s.t.} & & \left\{
                \begin{array}{ll}
                  \mathbb{P} \left(g_i(Z,d_1,d_2))\leq 0 \right) \leq P_{f}^T\\
                  -5 \leq d_1\leq 5 \\
                  -5 \leq d_2\leq 5 \\
                \end{array}
              \right.
\end{eqnarray}
with:
- $g_1(Z,\mathbf{d}) = (Z - d_2)-2*d_1$ 

and $Z$ a random Gaussian variable $Z\sim \mathcal{N}(0,0.5)$, $[p_1,p_2] = [\mu_{X_1},\mu_{X_2}]$ the mean values of the random variables $X_1,X_2$.
For the probability target maximal, it is assumed a reliability index $\beta = 2.$ leading to $P_f^T = \Phi(-\beta) \simeq 0.02275$ with $\Phi(\cdot)$ the CFD of a standard Normal distribution.

There is no $\mathbf{p}$ in this problem to be consistent with ot-robopt

In [16]:
# ot-robopt

J = ot.SymbolicFunction(
    ['d_1', 'd_2'], ['(d_1-0.25)^2 - d_2-d_1'])
calG = ot.SymbolicFunction(
    ['d_1', 'd_2', 'Z'], ['-((Z-d_2)-2*d_1)'])
g = ot.ParametricFunction(calG, [2], [0.])
reliabilityMeasure = otrobopt.JointChanceMeasure(g, ZDist, ot.Less(), 0.02275)
problem = otrobopt.RobustOptimizationProblem(J, reliabilityMeasure)
bounds = ot.Interval([-5.] * 2, [5.] * 2)
problem.setBounds(bounds)
algo = ot.Cobyla(problem)
t0 = time.time()
algo.setStartingPoint([0.0]*reliabilityMeasure.getInputDimension())
algo.run()
x_et = algo.getResult().getOptimalPoint()
print(x_et)
J_et = algo.getResult().getOptimalValue()
print(J_et)
res = algo.getResult()
print('Time CPU = ',time.time()-t0)

[-0.250005,-1.49999]
[2]
Time CPU =  0.1429729461669922


In [10]:
# RBDO RIA
## Definition of the objective Python function
def obj(d,p):
    d1 = d[0]
    d2 = d[1]        
    return (d1-0.25)**2-d1-d2

## Definition of the objective RBDO function with active d and p on the objective function. 
## In the example, no d are given (no active d), and the objective function depends on p=[p_1,p_2] (both active on the function),
## therefore in the vector XX=[d,p] p_1,p_2 are first and second index of XX
active_index_d = [0,1]
active_index_p = []
len_XX = 2
f_obj = Objective(obj,active_index_d,active_index_p,len_XX)

## Definition of the constraint functions
def g1(d,x,z):
    d1 = d[0]
    d2 = d[1]
    z0 = z[0]
    g1 = ((z0-d2)-2*d1)
    return [g1]

In [15]:
active_index_d_g1 = [0,1] 
active_index_p_g1 = [] 
PfT_g1 = 0.02275
PIneq_g1 = PIneqCons(g1,active_index_d_g1,active_index_p_g1,None,ZDist,PfT_g1)

## Definition of the list of constraints
PIneqCons_list = [PIneq_g1]  #list of Probability inequality constraints
DIneqCons_list = [] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([-5.,-5.], [5.,5.])  #Definition of the bounds on XX=[d,p] 
len_d = 2 #len of deterministic variable vector d
len_p = 0 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [0.,0.]

# Solver type of RIA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

t0 = time.time()
# Instantiation of the RIA algorithm
RIA_Algorithm_test = RIA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

# Run of RIA algorithm
result = RIA_Algorithm_test.run()

#Obtained results
print('Optimal design variables d = ',RIA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',RIA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',RIA_Algorithm_test.get_consoptimum())
print('Number of RIA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables d =  [-0.253461,-1.49308]
Optimal objective function value =  [2.00001]
Optimal constraint functions value =  [0.022749999996403505]
Number of RIA iterations =  37
Time CPU =  0.03716421127319336


In [14]:
##SORA 
InvFORM_solver_g1 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']

active_index_d_g1 = [0,1] 
active_index_p_g1 = [] 
PfT_g1 = 0.02275
PIneq_g1 = PIneqCons(g1,active_index_d_g1,active_index_p_g1,None,ZDist,PfT_g1,solver_invFORM = InvFORM_solver_g1) 

## Definition of the list of constraints
PIneqCons_list = [PIneq_g1]  #list of Probability inequality constraints
DIneqCons_list = [] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([-5.,-5.], [5.,5.])  #Definition of the bounds on XX=[d,p] 
len_d = 2 #len of deterministic variable vector d
len_p = 0 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [0.,0.]

# Solver type of SORA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

t0 = time.time()
# Instantiation of the SORA algorithm
SORA_Algorithm_test = SORA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

# Run of SORA algorithm
result = SORA_Algorithm_test.run()

#Obtained results
print('Optimal design variables d = ',SORA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',SORA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',SORA_Algorithm_test.get_consoptimum())
print('Number of SORA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)


Optimal design variables d =  [-0.249193,2.49839]
Optimal objective function value =  [-2]
Optimal constraint functions value =  [0.022750000000000006]
Number of SORA iterations =  31
Time CPU =  0.02968740463256836
